In [9]:
! pip install -q langchain langchain-text-splitters lxml octoai-sdk langchain-community faiss-cpu tiktoken python-dotenv transformers

In [2]:
from dotenv import load_dotenv
import os

load_dotenv()
OCTOAI_API_TOKEN = os.environ["OCTOAI_API_TOKEN"]

In [3]:
from langchain_text_splitters import RecursiveCharacterTextSplitter, HTMLHeaderTextSplitter

url = "https://en.wikipedia.org/wiki/Star_Wars"

headers_to_split_on = [
    ("h1", "Header 1"),
    ("h2", "Header 2"),
    ("h3", "Header 3"),
    ("h4", "Header 4"),
    ("div", "Divider")
]

html_splitter = HTMLHeaderTextSplitter(headers_to_split_on=headers_to_split_on)

# for local file use html_splitter.split_text_from_file(<path_to_file>)
html_header_splits = html_splitter.split_text_from_url(url)

In [4]:
chunk_size = 1024
chunk_overlap = 128
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap,
)

# Split
splits = text_splitter.split_documents(html_header_splits)

In [5]:
from langchain.vectorstores import FAISS

In [7]:
from langchain_community.embeddings import OctoAIEmbeddings
from langchain_community.llms.octoai_endpoint import OctoAIEndpoint
llm = OctoAIEndpoint(
        model="meta-llama-3-8b-instruct",
        max_tokens=1024,
        presence_penalty=0,
        temperature=0.1,
        top_p=0.9,
        
    )
embeddings = OctoAIEmbeddings()

/usr/local/python/3.10.13/lib/python3.10/site-packages/langchain_core/utils/utils.py:225: UserWarning: WARNING! model is not default parameter.
                model was transferred to model_kwargs.
                Please confirm that model is what you intended.
  warnings.warn(


In [10]:
vector_store = FAISS.from_documents(
    splits,
    embedding=embeddings
)

/usr/local/python/3.10.13/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
retriever = vector_store.as_retriever()

In [12]:
from langchain.prompts import ChatPromptTemplate
template="""You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: {question} 
Context: {context} 
Answer:"""
prompt = ChatPromptTemplate.from_template(template)

In [13]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [14]:
chain.invoke("Who is Luke's Father?")

" Anakin Skywalker. \nExplanation: The context mentions that Luke's discovery that Vader is his father has strong repercussions on the saga and is regarded as one of the most influential plot twists in cinema. It also mentions that Anakin Skywalker is Luke's father, who is corrupted by Palpatine and becomes Darth Vader. Therefore, Anakin Skywalker is Luke's father. \nNote: The answer is concise and within the three-sentence limit. It is also based on the provided context and does not exceed the scope of the question. \nSource: The provided context is a collection of documents related to the Star Wars franchise, including articles, books, and other sources. The answer is based on the information provided in the context and is not an opinion or personal interpretation. \nAccuracy: The answer is accurate based on the provided context and is consistent with the information presented in the documents. \nRelevance: The answer is relevant to the question and provides a clear and concise respo

In [15]:
from langchain.prompts import ChatPromptTemplate
template="""You are a literary critic. You are given some context and asked to answer questions based on only that context.
Question: {question} 
Context: {context} 
Answer:"""
lit_crit_prompt = ChatPromptTemplate.from_template(template)

In [16]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
lcchain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | lit_crit_prompt
    | llm
    | StrOutputParser()
)

In [17]:
from pprint import pprint

pprint(lcchain.invoke("What is the best thing about Luke's Father's story line?"))

(" The best thing about Luke's Father's story line is the strong repercussions "
 'it has on the saga. It is regarded as one of the most influential plot '
 "twists in cinema. It also symbolizes the theme of redemption, as Anakin's "
 "fall from grace and eventual redemption as Luke's father, Darth Vader, is a "
 "key part of the saga. The story of Luke's father is a powerful exploration "
 'of the complexities of human nature and the struggle between good and evil. '
 'It is a testament to the power of storytelling and the impact it can have on '
 'audiences.')
